_Feed Forward & Back-Propagation Learning Algorithm_

## Question 1

Implement the Perceptron algorithm from scratch in Python.
- Initialize the weights with [0 0 0] and a learning rate of 0.0001.
- For each iteration, calculate the output of the Perceptron for each input in the training set.
- Use MSE to computer the error for all samples
- Update the weights using the gradient descent procedure.
- Repeat the above steps until the Perceptron converges or a maximum number of iterations is reached.
- Test the trained Perceptron on a separate test set, explain how you came up with the test set.
- Use the step function as an  activation function in the output layer

Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class.

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Split randomly based on ratio 
def train_test_split(df,ratio):
    rows=df.shape[0]
    class_col=df.columns[-1]

    train_rows=int(rows*ratio)

    train=df.sample(n=train_rows)
    test=df[~df.isin(train).all(axis=1)]
    
    X_train=train.drop(class_col,axis=1)
    y_train=train[class_col]
    X_test=test.drop(class_col,axis=1)
    y_test=test[class_col]

    return X_train,y_train,X_test,y_test

class Perceptron():
    def __init__(self,l_rate):
        self.W=[]
        self.l_rate=l_rate

    def train(self,X_train,y_train):
        X_train=X_train.copy()
        y_train=y_train.copy()
        X_train['x0']=[1 for i in range(X_train.shape[0])]
        self.features=X_train.columns
        self.W=np.matrix([0 for i in range(len(self.features))])

        self.classes={}

        #Converting the classes to numerical values using a dictionary
        for i,c in enumerate(y_train.unique()):
            self.classes[i]=c

        y_train=y_train.replace(self.classes.values(),self.classes.keys()).astype(int)

        num_samples=X_train.shape[0]
        count=0
        MAX_ITERATIONS=200000
        iterations=0
        while count!=num_samples and iterations<MAX_ITERATIONS:
            x=np.matrix(X_train.iloc[count%num_samples])
            y_true=y_train.iloc[count%num_samples]
            y_pred=x*self.W.T
            if(y_pred<=0.5):
                y_pred=0
            else:
                y_pred=1
            if(y_pred!=y_true):
                #Correcting weight on misclassification
                self.W=self.W-(self.l_rate*np.multiply(x,(y_pred-y_true)))
                count=0
            count+=1
            iterations+=1

        print(f"Iterations: {iterations}")

    def predict(self,X_test):
        X_test=X_test.copy()
        y_pred=[]
        X_test['x0']=[1 for i in range(X_test.shape[0])]
        for i in X_test.index:
            x=np.matrix(X_test.loc[i])    
            temp=(x*self.W.T)
            if(temp<=0.5):
                y_pred.append(self.classes[0])
            else:
                y_pred.append(self.classes[1])
        return np.array(y_pred)

def accuracy(y_pred,y_test):
    count=0
    for i in range(y_pred.shape[0]):
        if(y_pred[i]==y_test.iloc[i]):
            count+=1
    return (count/y_pred.shape[0])*100

In [6]:
iris_df=pd.read_csv('iris.csv',index_col=0)
iris_df.drop(iris_df[iris_df['Species']=='Iris-virginica'].index,inplace=True)

X_train,y_train,X_test,y_test=train_test_split(iris_df,0.8)

model=Perceptron(0.0001)
model.train(X_train,y_train)
y_pred=model.predict(X_test)

print(f"Accuracy: {accuracy(y_pred,y_test)}%")

C:\Users\Falcon\AppData\Local\Temp\ipykernel_28536\707183073.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train=y_train.replace(self.classes.values(),self.classes.keys()).astype(int)


Iterations: 14831
Accuracy: 100.0%


## Question 2

Implement the feedforward and backpropagation learning algorithm for multi layer perceptrons in Python for the question provided in the attached image.
- Use  the weights and biases as given.
- Implement the forward pass.
- Compute the loss between the predicted output and the actual output using an appropriate loss function (MSE).
- Compute the gradients of the loss function with respect to the weights and biases using the chain rule.
- Update the weights and biases.
- Iterate over multiple times (epochs), performing forward propagation, loss calculation, backpropagation, and parameter updates in each iteration till convergence (the actual output is the same as the target output).


In [4]:
#A simple sigmoid function 
def sigmoid(x):
    return (1/(1+(np.e**(-x))))

'''NeuralNetwork Class
    #Inputs:
        - l_rate => Learing Rate
        - layers => Number of Layers
        - num_neurons => An array with the number of neurons in each layers
        - W  (optional) => A Matrix with the initial Weights / None
    #Member Variables:
        - l_rate => Learning Rate
        - layers => Number of layers
        - neurons => An array of neuron outputs of each layer
        - W => A matrix with the weights
        - errors => An array of errors from each layer
    #Methods:
        - status() => Print out the current state of the neural network
        - feed_forward(X) => Performs feedforward with X
        - backpropagate(y) => Performs backpropagation with y as the target output
        - output() => Returns the output in the output layer
        - train(X,y) => Train the model with iterations of feedforward and backpropagation
'''
class NeuralNetwork():
    def __init__(self,l_rate,layers,num_neurons,W=None):
        self.l_rate=l_rate
        self.layers=layers

        #Setting up the neuron matrix
        self.neurons=[]
        if(len(num_neurons)!=layers):
            raise Exception("Invalid number of layers/ Invald neurons array")
        for i in range(self.layers):
            layer_neurons=[]
            for j in range(num_neurons[i]):
                if(j==0 and i!=self.layers-1):
                    layer_neurons.append(1)
                else:
                    layer_neurons.append(0)
            self.neurons.append(np.array([layer_neurons]))

        #Setting up the weight matrices
        self.W=[]
        #If initial weights are not provided then default to an array of 0 matrix
        if not W:
            for i in range(self.layers-1):
                W_layer=[]
                for j in range(len(self.neurons[i+1][0])):
                    if(i!=self.layers-2 and j==0):
                        continue
                    W_neuron=[]
                    for k in range(len(self.neurons[i][0])):
                        W_neuron.append(0)
                        # print(f"Layer: {i} , Neurons:{k,j}")
                    W_layer.append(W_neuron)
                self.W.append(np.array(W_layer))
        #If the initial weights are given
        else:
            for i,W_layer in enumerate(W):
                W[i]=np.array(W_layer)
            self.W=W

        #Setting up the error matrices (used for backpropagation error lookup)
        self.errors=[]
        for i in range(1,self.layers):
            error_layer=[]
            for j in range(num_neurons[i]):
                if(i!=self.layers-1 and j==0):
                    continue
                error_layer.append(0)
            self.errors.append(np.array([error_layer]))

    #Print out the current state of the neural network
    def status(self):
        print(f"Layers: {self.layers}\n")
        print("Neurons:")
        display(self.neurons)
        print("Weights:")
        display(self.W)
        print("Errors:")
        display(self.errors)

    def feed_forward(self,X):
        self.neurons[0]=X
        for i in range(self.layers-1):
            outputs=[]
            if(i!=self.layers-2):
                outputs.append(1)
            outputs.extend(map(sigmoid,(self.neurons[i]@self.W[i].T)[0]))
            self.neurons[i+1]=np.array([outputs])
        return self.neurons[self.layers-1]

    def backpropagate(self,y):
        new_W=[]
        for i in range(self.layers-1,0,-1):
            new_W_layer=[]
            #If backpropagating from the output layer
            if(i==self.layers-1):
                self.errors[i-1]=self.neurons[i]*(self.neurons[i]-y)*(1-self.neurons[i])
            #If backpropagating from a hidden layer
            else:
                layer_doh=self.errors[i]@self.W[i]
                self.errors[i-1]=(self.neurons[i]*(1-self.neurons[i])*layer_doh)[:,1:]
                
            new_W_layer=self.W[i-1]-(self.l_rate*((self.errors[i-1].T)@(self.neurons[i-1])))
            new_W.append(np.array(new_W_layer))
        new_W.reverse()
        self.W=new_W

    #Returns the output in the output layer
    def output(self):
        return (self.neurons[self.layers-1])

    #Train the model with iterations of feedforward and backpropagation
    def train(self,X,y):
        MAX_EPOCHS=1000
        iterations=0
        for x in X:
            epoch=0
            while epoch<MAX_EPOCHS:
                y_pred=self.feed_forward(X)
                if((y_pred==y).all()):
                    break
                self.backpropagate(y)
                epoch+=1
                print(f"Epoch {epoch}: {self.output()}")
            print(f"\n\nEpochs: {epoch}")
            model.status()
            iterations+=1

In [7]:
model=NeuralNetwork(
    l_rate=0.5,
    layers=3,
    num_neurons=[3,3,2],
    W=[
        [[0.5,1.5,0.8],
         [0.8,0.2,-1.6]],
        [[0.9,-1.7,1.6],
         [1.2,2.1,-0.2]]
    ]
)

X=np.array([1,0.7,1.2]).reshape(1,-1)
y=np.array([1,0]).reshape(1,-1)

model.train(X,y)

Epoch 1: [[0.44137071 0.95637774]]
Epoch 2: [[0.48067847 0.95441807]]
Epoch 3: [[0.51841886 0.95229921]]
Epoch 4: [[0.55374605 0.95000577]]
Epoch 5: [[0.5861579  0.94751906]]
Epoch 6: [[0.61546946 0.94481632]]
Epoch 7: [[0.64173341 0.94187011]]
Epoch 8: [[0.66515038 0.93864764]]
Epoch 9: [[0.68599473 0.93510993]]
Epoch 10: [[0.70456357 0.93121083]]
Epoch 11: [[0.72114634 0.92689566]]
Epoch 12: [[0.73600942 0.92209952]]
Epoch 13: [[0.74939004 0.91674523]]
Epoch 14: [[0.76149554 0.91074071]]
Epoch 15: [[0.77250535 0.90397599]]
Epoch 16: [[0.7825741  0.89631949]]
Epoch 17: [[0.791835   0.88761395]]
Epoch 18: [[0.80040306 0.87767206]]
Epoch 19: [[0.80837788 0.86627211]]
Epoch 20: [[0.81584596 0.85315501]]
Epoch 21: [[0.82288243 0.83802396]]
Epoch 22: [[0.82955206 0.82054999]]
Epoch 23: [[0.8359097  0.80038766]]
Epoch 24: [[0.84199979 0.77720661]]
Epoch 25: [[0.84785531 0.75074467]]
Epoch 26: [[0.85349616 0.72088456]]
Epoch 27: [[0.85892779 0.68774583]]
Epoch 28: [[0.86414096 0.65176652]]
E

[array([[1. , 0.7, 1.2]]),
 array([[1.        , 0.77634647, 0.88540198]]),
 array([[0.98077526, 0.02073133]])]

Weights:


[array([[ 0.068086  ,  1.1976602 ,  0.2817032 ],
        [ 1.83237725,  0.92266408, -0.3611473 ]]),
 array([[ 2.2610751 , -0.53524045,  2.35716055],
        [-1.7620093 , -0.37921428, -2.03212936]])]

Errors:


[array([[ 5.99618989e-06, -1.73462890e-04]]),
 array([[-0.00036249,  0.00042088]])]